In [2]:
!pip install liac-arff

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=b26eaacb61c2bc3e755ceb3da722d4e8f969986b9a5ba2e36f87f105f2186ea4
  Stored in directory: /root/.cache/pip/wheels/00/23/31/5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff


In [3]:
import arff
import pandas as pd

# Load the ARFF file
with open('/content/Training Dataset.arff', 'r') as f:
    data = arff.load(f)

# Convert to DataFrame
df = pd.DataFrame(data['data'], columns=[attr[0] for attr in data['attributes']])

# Save to CSV
df.to_csv('/content/Training Dataset.cvs', index=False)


In [8]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("Training Dataset.cvs")  # Adjust filename/extension as needed

# Check the columns
print("Columns:", df.columns)

# Convert 'Request_URL' to string
df['Request_URL'] = df['Request_URL'].astype(str)

# Now split the 'Request_URL' column by comma into separate columns
split_cols = df['Request_URL'].str.split(',', expand=True)

# Optionally, rename the new columns
split_cols.columns = [f"Request_URL_part_{i}" for i in range(split_cols.shape[1])]

# Drop the original 'Request_URL' column and join the new columns
df = df.drop('Request_URL', axis=1).join(split_cols)

# Save the new DataFrame to a CSV file
df.to_csv("Training_Dataset_split.csv", index=False)
print("Data saved to 'Training_Dataset_split.csv'.")


Columns: Index(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Submitting_to_email', 'Abnormal_URL',
       'Redirect', 'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe',
       'age_of_domain', 'DNSRecord', 'web_traffic', 'Page_Rank',
       'Google_Index', 'Links_pointing_to_page', 'Statistical_report',
       'Result'],
      dtype='object')
Data saved to 'Training_Dataset_split.csv'.


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# For reproducibility
RANDOM_STATE = 42

In [ ]:
# Load the dataset. 
df = pd.read_csv("/content/URL_Dataset.cvs")  
print("Dataset loaded successfully!")
print("Shape:", df.shape)
print("Columns:", df.columns)
df.head()

Dataset loaded successfully!
Shape: (11055, 31)
Columns: Index(['having_IP_Address', 'URL_Length', 'Shortining_Service',
       'having_At_Symbol', 'double_slash_redirecting', 'Prefix_Suffix',
       'having_Sub_Domain', 'SSLfinal_State', 'Domain_registeration_length',
       'Favicon', 'port', 'HTTPS_token', 'URL_of_Anchor', 'Links_in_tags',
       'SFH', 'Submitting_to_email', 'Abnormal_URL', 'Redirect',
       'on_mouseover', 'RightClick', 'popUpWidnow', 'Iframe', 'age_of_domain',
       'DNSRecord', 'web_traffic', 'Page_Rank', 'Google_Index',
       'Links_pointing_to_page', 'Statistical_report', 'Result',
       'Request_URL_part_0'],
      dtype='object')


,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result,Request_URL_part_0
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,-1,-1,-1,-1,1,1,-1,-1,1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,-1,-1,0,-1,1,1,1,-1,1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,-1,1,-1,1,0,-1,-1,1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,-1,-1,1,-1,1,-1,1,-1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,1,-1,-1,0,-1,1,1,1,1,1


In [13]:
# Data Exploration
print(df.describe())
print("\nMissing values per column:")
print(df.isnull().sum())

       having_IP_Address    URL_Length  Shortining_Service  having_At_Symbol  \
count       11055.000000  11055.000000        11055.000000      11055.000000   
mean            0.313795     -0.633198            0.738761          0.700588   
std             0.949534      0.766095            0.673998          0.713598   
min            -1.000000     -1.000000           -1.000000         -1.000000   
25%            -1.000000     -1.000000            1.000000          1.000000   
50%             1.000000     -1.000000            1.000000          1.000000   
75%             1.000000     -1.000000            1.000000          1.000000   
max             1.000000      1.000000            1.000000          1.000000   

       double_slash_redirecting  Prefix_Suffix  having_Sub_Domain  \
count              11055.000000   11055.000000       11055.000000   
mean                   0.741474      -0.734962           0.063953   
std                    0.671011       0.678139           0.817518   
min

In [14]:
# Check the data types and convert if needed
print("Data types:")
print(df.dtypes)

Data types:
having_IP_Address              int64
URL_Length                     int64
Shortining_Service             int64
having_At_Symbol               int64
double_slash_redirecting       int64
Prefix_Suffix                  int64
having_Sub_Domain              int64
SSLfinal_State                 int64
Domain_registeration_length    int64
Favicon                        int64
port                           int64
HTTPS_token                    int64
URL_of_Anchor                  int64
Links_in_tags                  int64
SFH                            int64
Submitting_to_email            int64
Abnormal_URL                   int64
Redirect                       int64
on_mouseover                   int64
RightClick                     int64
popUpWidnow                    int64
Iframe                         int64
age_of_domain                  int64
DNSRecord                      int64
web_traffic                    int64
Page_Rank                      int64
Google_Index              

In [15]:
# Assume the target column is named 'Result' (0 for safe, 1 for malicious)
if 'Result' not in df.columns:
    raise ValueError("The dataset must contain a 'Result' column as the target variable.")


In [16]:
# For this example, we'll assume that all columns except 'Result' are features.
X = df.drop("Result", axis=1)
y = df["Result"]

In [17]:
# Optionally, fill missing values (if any) with the median of each column
X = X.fillna(X.median())

In [18]:
print("Preprocessed features shape:", X.shape)
print("Target distribution:")
print(y.value_counts())

Preprocessed features shape: (11055, 30)
Target distribution:
Result
 1    6157
-1    4898
Name: count, dtype: int64


In [19]:
# Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (8844, 30)
Test set shape: (2211, 30)


# ## Machine Learning Models
#
# We'll train several models and compare their performance:
#
# 1. Logistic Regression  
# 2. Decision Tree  
# 3. Random Forest  
# 4. Support Vector Machine (SVM)  
# 5. K-Nearest Neighbors (KNN)  
# 6. Naive Bayes  
# 7. Gradient Boosting  
# 8. XGBoost  
# 9. Stacking Classifier


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

In [ ]:
def evaluate_model(model, X_test, y_test):
    from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    try:
        y_proba = model.predict_proba(X_test)[:, 1]
        print("ROC AUC:", roc_auc_score(y_test, y_proba))
    except Exception as e:
        print("ROC AUC cannot be computed:", e)
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    print("\n")

In [ ]:
# 1. Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train, y_train)
print("Logistic Regression Performance:")
evaluate_model(lr, X_test, y_test)

Logistic Regression Performance:
Accuracy: 0.924468566259611
ROC AUC: 0.9770291220057011
Classification Report:
              precision    recall  f1-score   support

          -1       0.92      0.90      0.91       956
           1       0.93      0.94      0.93      1255

    accuracy                           0.92      2211
   macro avg       0.92      0.92      0.92      2211
weighted avg       0.92      0.92      0.92      2211





In [ ]:
# 2. Decision Tree
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
dt.fit(X_train, y_train)
print("Decision Tree Performance:")
evaluate_model(dt, X_test, y_test)

Decision Tree Performance:
Accuracy: 0.9579375848032564
ROC AUC: 0.9709075830568938
Classification Report:
              precision    recall  f1-score   support

          -1       0.95      0.95      0.95       956
           1       0.96      0.96      0.96      1255

    accuracy                           0.96      2211
   macro avg       0.96      0.96      0.96      2211
weighted avg       0.96      0.96      0.96      2211





In [ ]:
# 3. Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
print("Random Forest Performance:")
evaluate_model(rf, X_test, y_test)

Random Forest Performance:
Accuracy: 0.966078697421981
ROC AUC: 0.9957492206904599
Classification Report:
              precision    recall  f1-score   support

          -1       0.97      0.95      0.96       956
           1       0.96      0.98      0.97      1255

    accuracy                           0.97      2211
   macro avg       0.97      0.96      0.97      2211
weighted avg       0.97      0.97      0.97      2211





In [ ]:
# 4. Support Vector Machine (SVM)
# We'll use probability=True to compute ROC AUC.
svm = SVC(probability=True, random_state=RANDOM_STATE)
svm.fit(X_train, y_train)
print("SVM Performance:")
evaluate_model(svm, X_test, y_test)

SVM Performance:
Accuracy: 0.9470827679782904
ROC AUC: 0.9862208071479771
Classification Report:
              precision    recall  f1-score   support

          -1       0.95      0.92      0.94       956
           1       0.94      0.97      0.95      1255

    accuracy                           0.95      2211
   macro avg       0.95      0.94      0.95      2211
weighted avg       0.95      0.95      0.95      2211





In [ ]:
# 5. K-Nearest Neighbors (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print("KNN Performance:")
evaluate_model(knn, X_test, y_test)

KNN Performance:
Accuracy: 0.9407507914970602
ROC AUC: 0.9819616929770458
Classification Report:
              precision    recall  f1-score   support

          -1       0.94      0.92      0.93       956
           1       0.94      0.95      0.95      1255

    accuracy                           0.94      2211
   macro avg       0.94      0.94      0.94      2211
weighted avg       0.94      0.94      0.94      2211





In [ ]:
# 6. Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)
print("Naive Bayes Performance:")
evaluate_model(nb, X_test, y_test)

Naive Bayes Performance:
Accuracy: 0.5829941203075532
ROC AUC: 0.965533681174882
Classification Report:
              precision    recall  f1-score   support

          -1       0.51      1.00      0.67       956
           1       1.00      0.27      0.42      1255

    accuracy                           0.58      2211
   macro avg       0.75      0.63      0.55      2211
weighted avg       0.79      0.58      0.53      2211





In [ ]:
# 7. Gradient Boosting
gb = GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE)
gb.fit(X_train, y_train)
print("Gradient Boosting Performance:")
evaluate_model(gb, X_test, y_test)

Gradient Boosting Performance:
Accuracy: 0.9507010402532791
ROC AUC: 0.9902482121722316
Classification Report:
              precision    recall  f1-score   support

          -1       0.95      0.93      0.94       956
           1       0.95      0.96      0.96      1255

    accuracy                           0.95      2211
   macro avg       0.95      0.95      0.95      2211
weighted avg       0.95      0.95      0.95      2211





In [ ]:
# Assume df is your DataFrame and 'Result' is your target column
X = df.drop("Result", axis=1)
y = df["Result"]

# Convert target labels to 0 and 1
y = y.map({-1: 0, 1: 1})

# Now perform train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)


In [ ]:
# 8. XGBoost Classifier
import xgboost as xgb
RANDOM_STATE = 42
xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1,
                            use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE)
xgb_clf.fit(X_train, y_train)
print("XGBoost Performance:")
evaluate_model(xgb_clf, X_test, y_test)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:53:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Performance:
Accuracy: 0.9601990049751243
ROC AUC: 0.9936621713980898
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       956
           1       0.96      0.97      0.97      1255

    accuracy                           0.96      2211
   macro avg       0.96      0.96      0.96      2211
weighted avg       0.96      0.96      0.96      2211





In [ ]:
# 9. Stacking Classifier
estimators = [
    ('lr', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)),
    ('svm', SVC(probability=True, random_state=RANDOM_STATE)),
    ('xgb', xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE))
]

In [ ]:
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=GradientBoostingClassifier(random_state=RANDOM_STATE))
stacking_clf.fit(X_train, y_train)
print("Stacking Classifier Performance:")
evaluate_model(stacking_clf, X_test, y_test)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:54:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:54:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:54:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:54:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:54:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

Stacking Classifier Performance:
Accuracy: 0.9674355495251018
ROC AUC: 0.995745470002834
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       956
           1       0.96      0.98      0.97      1255

    accuracy                           0.97      2211
   macro avg       0.97      0.97      0.97      2211
weighted avg       0.97      0.97      0.97      2211





In [ ]:
import pickle

# Save the trained XGBoost classifier
with open("XGBoostClassifier.pkl", "wb") as f:
    pickle.dump(xgb_clf, f)

print("Model saved as XGBoostClassifier.pkl")


Model saved as XGBoostClassifier.pkl


In [ ]:
import pickle

with open("XGBoostClassifier.pkl", "rb") as f:
    xgb_clf_loaded = pickle.load(f)

# Now you can use xgb_clf_loaded for predictions


In [ ]:
# Save the booster from your XGBClassifier model
xgb_clf.get_booster().save_model("XGBoostClassifier.json")
print("Model saved as XGBoostClassifier.json")


Model saved as XGBoostClassifier.json


In [ ]:
import xgboost as xgb

model_booster = xgb.Booster()
model_booster.load_model("XGBoostClassifier.json")
# Use model_booster for predictions via a DMatrix:
# e.g., model_booster.predict(xgb.DMatrix(X_test))
